# EDA


In [8]:
# importing libraries required
import pandas as pd
import numpy as np
import seaborn as sns

In [9]:
!dir

 Volume in drive D is Window
 Volume Serial Number is 2C5A-22B4

 Directory of D:\analytics_vidhya\demand_forecasting

08/07/2021  10:33 PM    <DIR>          .
08/07/2021  10:33 PM    <DIR>          ..
08/07/2021  01:56 PM    <DIR>          .ipynb_checkpoints
08/07/2021  10:33 PM            56,743 forecasting.ipynb
08/07/2021  01:48 PM           325,744 sample_submission_hSlSoT6.csv
08/07/2021  07:53 PM           410,926 submission.csv
08/07/2021  01:48 PM         1,239,576 test_QoiMO9B.csv
08/07/2021  01:48 PM    <DIR>          train_GzS76OK
08/07/2021  01:48 PM         5,487,936 train_GzS76OK.zip
               5 File(s)      7,520,925 bytes
               4 Dir(s)  554,132,815,872 bytes free


In [10]:
train = pd.read_csv("train_GzS76OK/train.csv")
test = pd.read_csv("test_QoiMO9B.csv")
meal_info = pd.read_csv("train_GzS76OK/meal_info.csv")
fulfill = pd.read_csv("train_GzS76OK/fulfilment_center_info.csv")

In [11]:
train = pd.merge(train,meal_info,on='meal_id',how='left')
train = pd.merge(train,fulfill,on='center_id',how='left')

In [12]:
test = pd.merge(test,meal_info,on='meal_id',how='left')
test = pd.merge(test,fulfill,on='center_id',how='left')

In [13]:


train['category'] = train['category'].astype('category')
train['cuisine'] = train['cuisine'].astype('category')
train['center_type'] = train['center_type'].astype('category')
test['category'] = test['category'].astype('category')
test['cuisine'] = test['cuisine'].astype('category')
test['center_type'] = test['center_type'].astype('category')
category_cols = ['category','cuisine','center_type']
for c in category_cols:
    train[c] = train[c].cat.codes
    test[c] = test[c].cat.codes

In [14]:
train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders', 'category',
       'cuisine', 'city_code', 'region_code', 'center_type', 'op_area'],
      dtype='object')

# Feature selection

In [15]:
features_col = ['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'category',
       'cuisine', 'city_code', 'region_code', 'center_type', 'op_area']


In [23]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(train, test_size=0.30, random_state = 5)


# Training and Cross validation

In [24]:
from sklearn.linear_model import LinearRegression

In [25]:
reg = LinearRegression().fit(train[features_col], train['num_orders'])

In [39]:
predict_val = reg.predict(validation[features_col])


In [41]:
from sklearn.metrics import mean_squared_log_error
predict_val = np.abs(predict_val)
mean_squared_log_error(validation['num_orders'],predict_val)**0.5

0.5308034149581525

In [36]:
import lightgbm as lgb
lgb_train = lgb.Dataset(train[features_col], train['num_orders'])
# lgb_eval = lgb.Dataset(train[features_col], train['num_orders'], reference=lgb_train)
params = {'boosting_type': 'gbdt',
           'objective': 'regression',
           'num_leaves': 200,
           'learning_rate': 0.1,
           'feature_fraction': 0.9}
gbm = lgb.train(params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train],
    valid_names=['train'],
   )

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1146
[LightGBM] [Info] Number of data points in the train set: 223708, number of used features: 14
[LightGBM] [Info] Start training from score 262.290173
[1]	train's l2: 137476
[2]	train's l2: 120706
[3]	train's l2: 105705
[4]	train's l2: 93632.6
[5]	train's l2: 83506.7
[6]	train's l2: 75226.6
[7]	train's l2: 68157.8
[8]	train's l2: 62299.1
[9]	train's l2: 57300.4
[10]	train's l2: 52975.3
[11]	train's l2: 49295.1
[12]	train's l2: 46215.7
[13]	train's l2: 43504.5
[14]	train's l2: 41073.3
[15]	train's l2: 38887.1
[16]	train's l2: 37103.7
[17]	train's l2: 35537.1
[18]	train's l2: 34071.9
[19]	train's l2: 32921.7
[20]	train's l2: 31807.9
[21]	train's l2: 30863.9
[22]	train's l2: 29991.9
[23]	train's l2: 29150.2
[24]	train's l2: 28396.3
[25]	train's l2: 27742.3
[26]	train's l2: 27057.5
[27]	train's l

[315]	train's l2: 8101.33
[316]	train's l2: 8072.1
[317]	train's l2: 8051.29
[318]	train's l2: 8035.89
[319]	train's l2: 8021.9
[320]	train's l2: 7992.79
[321]	train's l2: 7969.43
[322]	train's l2: 7947.5
[323]	train's l2: 7928.28
[324]	train's l2: 7923.6
[325]	train's l2: 7912.53
[326]	train's l2: 7898.34
[327]	train's l2: 7886.63
[328]	train's l2: 7879.76
[329]	train's l2: 7862.34
[330]	train's l2: 7848.55
[331]	train's l2: 7822.76
[332]	train's l2: 7810.81
[333]	train's l2: 7802.76
[334]	train's l2: 7782.91
[335]	train's l2: 7759.36
[336]	train's l2: 7741.28
[337]	train's l2: 7730.71
[338]	train's l2: 7722.26
[339]	train's l2: 7718.65
[340]	train's l2: 7699.32
[341]	train's l2: 7675.84
[342]	train's l2: 7666.76
[343]	train's l2: 7662.5
[344]	train's l2: 7650.93
[345]	train's l2: 7634.58
[346]	train's l2: 7626.64
[347]	train's l2: 7613.36
[348]	train's l2: 7601.55
[349]	train's l2: 7587.54
[350]	train's l2: 7577.15
[351]	train's l2: 7566.61
[352]	train's l2: 7558.95
[353]	train's l2:

[644]	train's l2: 5106.05
[645]	train's l2: 5100.55
[646]	train's l2: 5095.62
[647]	train's l2: 5089.62
[648]	train's l2: 5084.95
[649]	train's l2: 5083.06
[650]	train's l2: 5080.1
[651]	train's l2: 5075.22
[652]	train's l2: 5070.15
[653]	train's l2: 5064.14
[654]	train's l2: 5060.22
[655]	train's l2: 5054.91
[656]	train's l2: 5049.76
[657]	train's l2: 5045.9
[658]	train's l2: 5036.71
[659]	train's l2: 5029.64
[660]	train's l2: 5027.23
[661]	train's l2: 5022.68
[662]	train's l2: 5017.52
[663]	train's l2: 5013.65
[664]	train's l2: 5008.63
[665]	train's l2: 5004.07
[666]	train's l2: 5000.3
[667]	train's l2: 4994.86
[668]	train's l2: 4990.34
[669]	train's l2: 4988.66
[670]	train's l2: 4980.3
[671]	train's l2: 4976.97
[672]	train's l2: 4971.62
[673]	train's l2: 4966.75
[674]	train's l2: 4961.61
[675]	train's l2: 4957.2
[676]	train's l2: 4952.94
[677]	train's l2: 4948.9
[678]	train's l2: 4941.3
[679]	train's l2: 4936.72
[680]	train's l2: 4927.79
[681]	train's l2: 4922.99
[682]	train's l2: 4

[983]	train's l2: 3745.21
[984]	train's l2: 3742.08
[985]	train's l2: 3739.24
[986]	train's l2: 3734.71
[987]	train's l2: 3732.53
[988]	train's l2: 3728.24
[989]	train's l2: 3725.42
[990]	train's l2: 3724.46
[991]	train's l2: 3722.46
[992]	train's l2: 3716.9
[993]	train's l2: 3713.88
[994]	train's l2: 3710.56
[995]	train's l2: 3708.21
[996]	train's l2: 3705.23
[997]	train's l2: 3701.86
[998]	train's l2: 3698.46
[999]	train's l2: 3695.25
[1000]	train's l2: 3690.83


In [40]:
predict_val = gbm.predict(validation[features_col])

In [15]:
submission = pd.concat([test['id'],pd.Series(predictions)],axis=1)
submission = submission.rename(columns={0:'num_orders'})
submission['num_orders'] = submission['num_orders'].astype(int)
submission['num_orders'] = submission['num_orders'].apply(abs)
submission.to_csv('submission.csv',index=False)

In [12]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [ 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, ],
             'reg_lambda': [0, ]}

In [11]:
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=50)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

NameError: name 'param_test' is not defined

In [51]:
param_test

{'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen at 0x1ca89108700>,
 'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen at 0x1ca801259d0>,
 'min_child_weight': [10000.0],
 'subsample': <scipy.stats._distn_infrastructure.rv_frozen at 0x1ca88b3f460>,
 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen at 0x1ca8420c880>,
 'reg_alpha': [0],
 'reg_lambda': [0]}

In [26]:
from sklearn import svm
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


model_params = {
    'random_forest': {
        'model': lgb.LGBMClassifier(),
        'params' : {
            'n_estimators': [1,5,10],
            'objective':['regression'],
            'num_leaves':[200],
            'learning_rate': [0.1],
           'feature_fraction':[ 0.9]         
        }
    },
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(train[features_col], train['num_orders'])
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

c:\users\rishikesh.kushwaha\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGB